# 지하철 1호선 복합 지표 분석
## 소요시간 × 배차 간격 = 승객 대기 시간 지표

이 분석은 **역간 소요시간**과 **배차 간격**을 결합하여 승객의 실제 경험을 정량화합니다.

### 핵심 개념
```
복합 지표 = 평균 소요시간 × 평균 배차 간격
```

**의미**:
- 소요시간이 길다 → 이동에 오래 걸림
- 배차 간격이 길다 → 열차 대기 시간이 김
- 둘 다 크면 → 승객 불편이 큼 (혼잡도 높을 가능성)

**활용**:
- 서비스 개선이 필요한 구간 식별
- 배차 증편이 필요한 구간 우선순위 결정
- 혼잡도 예측 모델의 입력 변수

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sqlite3
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

DB_PATH = r"..\data\input\metro_datetime.db"
TABLE = "Metro_Line_1_Schedule"

## 1. 전체 데이터 로드 및 전처리

In [2]:
# 데이터 로드
with sqlite3.connect(DB_PATH) as conn:
    query = f"""
        SELECT 
            `역사코드`,
            `역사명`,
            `주중주말`,
            `방향`,
            `급행여부`,
            `열차코드`,
            `열차도착시간_dt`,
            `열차출발시간_dt`,
            `출발역`,
            `도착역`
        FROM {TABLE}
        WHERE `급행여부` = 0
        ORDER BY `열차코드`, `열차도착시간_dt`
    """
    df_all = pd.read_sql(query, conn)

# datetime 변환
df_all['열차도착시간_dt'] = pd.to_datetime(df_all['열차도착시간_dt'])
df_all['열차출발시간_dt'] = pd.to_datetime(df_all['열차출발시간_dt'])

print(f"📊 전체 데이터: {len(df_all):,}행")
print(f"🚇 고유 열차 수: {df_all['열차코드'].nunique():,}대")
print(f"🚉 고유 역 수: {df_all['역사명'].nunique():,}개")
print(f"📅 주중주말: {df_all['주중주말'].unique()}")

display(df_all.head())

📊 전체 데이터: 68,791행
🚇 고유 열차 수: 708대
🚉 고유 역 수: 102개
📅 주중주말: ['DAY' 'END' 'SAT']


,역사코드,역사명,주중주말,방향,급행여부,열차코드,열차도착시간_dt,열차출발시간_dt,출발역,도착역
0,1701,구로,DAY,DOWN,0,K1,NaT,2025-07-04 05:00:00,구로,인천
1,1701,구로,END,DOWN,0,K1,NaT,2025-07-04 05:00:00,구로,인천
2,1701,구로,SAT,DOWN,0,K1,NaT,2025-07-04 05:00:00,구로,인천
3,1813,구일,DAY,DOWN,0,K1,2025-07-04 05:02:00,2025-07-04 05:02:30,구로,인천
4,1813,구일,END,DOWN,0,K1,2025-07-04 05:02:00,2025-07-04 05:02:30,구로,인천


## 2. 역간 소요시간 계산

각 열차의 연속된 정차역 사이의 이동 시간을 계산합니다.

In [3]:
def calculate_inter_station_time(df):
    """
    역간 소요시간 계산
    """
    travel_times = []
    
    for (train_code, day_type), group in df.groupby(['열차코드', '주중주말']):
        group = group.sort_values('열차도착시간_dt').reset_index(drop=True)
        
        for i in range(len(group) - 1):
            current = group.iloc[i]
            next_station = group.iloc[i + 1]
            
            if pd.notna(current['열차출발시간_dt']) and pd.notna(next_station['열차도착시간_dt']):
                travel_time = (next_station['열차도착시간_dt'] - current['열차출발시간_dt']).total_seconds() / 60
                
                if 0 < travel_time < 60:
                    travel_times.append({
                        '출발역명': current['역사명'],
                        '도착역명': next_station['역사명'],
                        '출발역코드': current['역사코드'],
                        '도착역코드': next_station['역사코드'],
                        '소요시간_분': travel_time,
                        '열차코드': train_code,
                        '주중주말': day_type,
                        '방향': current['방향']
                    })
    
    return pd.DataFrame(travel_times)

print("⏱️ 역간 소요시간 계산 중...")
df_travel = calculate_inter_station_time(df_all)

# 구간별 평균 소요시간
travel_stats = df_travel.groupby(['출발역명', '도착역명', '방향', '주중주말']).agg({
    '소요시간_분': ['mean', 'std', 'count']
}).round(2)
travel_stats.columns = ['평균소요시간_분', '표준편차', '측정횟수']
travel_stats = travel_stats.reset_index()

print(f"✅ {len(df_travel):,}개의 역간 이동 기록")
print(f"✅ {len(travel_stats):,}개의 고유 구간 (방향별, 주중주말별)")

display(travel_stats.head(20))

⏱️ 역간 소요시간 계산 중...
✅ 65,526개의 역간 이동 기록
✅ 595개의 고유 구간 (방향별, 주중주말별)


,출발역명,도착역명,방향,주중주말,평균소요시간_분,표준편차,측정횟수
0,가능,녹양,UP,DAY,1.5,0.0,96
1,가능,녹양,UP,END,1.5,0.0,88
2,가능,녹양,UP,SAT,1.5,0.0,88
3,가능,의정부,DOWN,DAY,2.0,0.0,96
4,가능,의정부,DOWN,END,2.0,0.0,88
5,가능,의정부,DOWN,SAT,2.0,0.0,88
6,가산디지털단지,구로,UP,DAY,4.0,0.0,131
7,가산디지털단지,구로,UP,END,4.0,0.0,126
8,가산디지털단지,구로,UP,SAT,4.0,0.0,126
9,가산디지털단지,독산,DOWN,DAY,2.0,0.0,129


## 3. 배차 간격(Headway) 계산

각 역에서 연속된 열차가 도착하는 시간 간격을 계산합니다.

In [4]:
def calculate_headway_by_station(df):
    """
    역별, 방향별, 주중주말별 배차 간격 계산
    """
    headways = []
    
    for (station, direction, day_type), group in df.groupby(['역사명', '방향', '주중주말']):
        # 도착시간이 있는 데이터만
        group = group[group['열차도착시간_dt'].notna()].copy()
        group = group.sort_values('열차도착시간_dt').reset_index(drop=True)
        
        if len(group) > 1:
            # 배차 간격 계산
            group['배차간격_분'] = group['열차도착시간_dt'].diff().dt.total_seconds() / 60
            
            # 첫 번째 행 제외 (diff로 인한 NaN)
            valid_headways = group['배차간격_분'].dropna()
            
            # 이상치 제거 (1분~60분 범위)
            valid_headways = valid_headways[(valid_headways > 1) & (valid_headways < 60)]
            
            if len(valid_headways) > 0:
                headways.append({
                    '역사명': station,
                    '역사코드': group.iloc[0]['역사코드'],
                    '방향': direction,
                    '주중주말': day_type,
                    '평균배차간격_분': valid_headways.mean(),
                    '최소배차간격_분': valid_headways.min(),
                    '최대배차간격_분': valid_headways.max(),
                    '표준편차': valid_headways.std(),
                    '측정횟수': len(valid_headways)
                })
    
    return pd.DataFrame(headways)

print("⏱️ 배차 간격 계산 중...")
df_headway = calculate_headway_by_station(df_all)

print(f"\n✅ {len(df_headway):,}개의 역-방향-주중주말 조합")
print(f"\n평균 배차 간격: {df_headway['평균배차간격_분'].mean():.2f}분")

display(df_headway.head(20))

⏱️ 배차 간격 계산 중...

✅ 597개의 역-방향-주중주말 조합

평균 배차 간격: 14.10분


,역사명,역사코드,방향,주중주말,평균배차간격_분,최소배차간격_분,최대배차간격_분,표준편차,측정횟수
0,가능,1907,DOWN,DAY,12.068421,3.5,32.5,6.164677,95
1,가능,1907,DOWN,END,13.086207,5.0,26.0,5.224165,87
2,가능,1907,DOWN,SAT,13.086207,5.0,26.0,5.224165,87
3,가능,1907,UP,DAY,11.863158,3.0,27.0,5.518558,95
4,가능,1907,UP,END,12.988506,4.0,26.0,5.196699,87
5,가능,1907,UP,SAT,12.988506,4.0,26.0,5.196699,87
6,가산디지털단지,1702,DOWN,DAY,8.738281,3.0,20.0,3.817162,128
7,가산디지털단지,1702,DOWN,END,8.964000,3.0,21.0,3.965159,125
8,가산디지털단지,1702,DOWN,SAT,8.964000,3.0,21.0,3.965159,125
9,가산디지털단지,1702,UP,DAY,8.607692,3.5,20.0,3.510772,130


## 4. 복합 지표 계산: 소요시간 × 배차 간격

각 구간의 소요시간과 출발역의 배차 간격을 곱하여 복합 지표를 생성합니다.

In [5]:
# 역간 소요시간에 출발역의 배차 간격 결합
df_combined = travel_stats.merge(
    df_headway[['역사명', '방향', '주중주말', '평균배차간격_분']],
    left_on=['출발역명', '방향', '주중주말'],
    right_on=['역사명', '방향', '주중주말'],
    how='left'
)

# 불필요한 컬럼 제거
df_combined = df_combined.drop('역사명', axis=1)

# 복합 지표 계산
df_combined['복합지표'] = df_combined['평균소요시간_분'] * df_combined['평균배차간격_분']

# 구간명 생성
df_combined['구간'] = df_combined['출발역명'] + ' → ' + df_combined['도착역명']

# 결측값 제거 (배차 간격 데이터가 없는 경우)
df_combined = df_combined.dropna(subset=['평균배차간격_분', '복합지표'])

print(f"✅ 복합 지표 계산 완료: {len(df_combined):,}개 구간")
print(f"\n복합 지표 통계:")
print(f"  평균: {df_combined['복합지표'].mean():.2f}")
print(f"  중앙값: {df_combined['복합지표'].median():.2f}")
print(f"  최소: {df_combined['복합지표'].min():.2f}")
print(f"  최대: {df_combined['복합지표'].max():.2f}")

display(df_combined.head(20))

✅ 복합 지표 계산 완료: 593개 구간

복합 지표 통계:
  평균: 38.27
  중앙값: 20.90
  최소: 5.42
  최대: 419.25


,출발역명,도착역명,방향,주중주말,평균소요시간_분,표준편차,측정횟수,평균배차간격_분,복합지표,구간
0,가능,녹양,UP,DAY,1.5,0.0,96,11.863158,17.794737,가능 → 녹양
1,가능,녹양,UP,END,1.5,0.0,88,12.988506,19.482759,가능 → 녹양
2,가능,녹양,UP,SAT,1.5,0.0,88,12.988506,19.482759,가능 → 녹양
3,가능,의정부,DOWN,DAY,2.0,0.0,96,12.068421,24.136842,가능 → 의정부
4,가능,의정부,DOWN,END,2.0,0.0,88,13.086207,26.172414,가능 → 의정부
5,가능,의정부,DOWN,SAT,2.0,0.0,88,13.086207,26.172414,가능 → 의정부
6,가산디지털단지,구로,UP,DAY,4.0,0.0,131,8.607692,34.430769,가산디지털단지 → 구로
7,가산디지털단지,구로,UP,END,4.0,0.0,126,8.944000,35.776000,가산디지털단지 → 구로
8,가산디지털단지,구로,UP,SAT,4.0,0.0,126,8.944000,35.776000,가산디지털단지 → 구로
9,가산디지털단지,독산,DOWN,DAY,2.0,0.0,129,8.738281,17.476562,가산디지털단지 → 독산


## 5. 복합 지표 Top 50 분석 (주중)

승객 불편이 가장 큰 구간을 식별합니다.

In [6]:
# 주중 데이터만 필터링
df_weekday = df_combined[df_combined['주중주말'] == 'DAY'].copy()

# 복합 지표 큰 순서로 정렬
df_weekday_sorted = df_weekday.sort_values('복합지표', ascending=False)

print("🚨 복합 지표 Top 50 (주중) - 승객 대기 시간이 긴 구간")
print("=" * 120)
print("복합지표가 크다 = 소요시간도 길고 + 배차 간격도 길다 = 승객 불편이 큼")
print("=" * 120)

# Top 50 표시
top50_weekday = df_weekday_sorted.head(50)[[
    '구간', '방향', '평균소요시간_분', '평균배차간격_분', '복합지표', '측정횟수'
]].reset_index(drop=True)

display(top50_weekday)

# CSV 저장
top50_weekday.to_csv('..\data\output\복합지표_Top50_주중.csv', index=False, encoding='utf-8-sig')
print("\n✅ '복합지표_Top50_주중.csv' 파일로 저장되었습니다.")

🚨 복합 지표 Top 50 (주중) - 승객 대기 시간이 긴 구간
복합지표가 크다 = 소요시간도 길고 + 배차 간격도 길다 = 승객 불편이 큼


,구간,방향,평균소요시간_분,평균배차간격_분,복합지표,측정횟수
0,전곡 → 연천,UP,7.50,48.961538,367.211538,21
1,청산 → 소요산,DOWN,5.50,49.000000,269.500000,21
2,전곡 → 청산,DOWN,4.50,49.000000,220.500000,21
3,소요산 → 동두천,DOWN,3.50,49.000000,171.500000,21
4,청산 → 전곡,UP,3.00,48.923077,146.769231,21
5,소요산 → 청산,UP,4.50,30.256757,136.155405,21
6,온양온천 → 신창,DOWN,4.50,29.375000,132.187500,33
7,천안 → 두정,UP,4.03,31.586207,127.292414,32
8,온양온천 → 배방,UP,4.00,31.586207,126.344828,33
9,평택 → 성환,DOWN,6.49,18.508333,120.119083,61



✅ '복합지표_Top50_주중.csv' 파일로 저장되었습니다.


## 6. 복합 지표 Top 50 분석 (주말)

In [7]:
# 주말 데이터
df_weekend = df_combined[df_combined['주중주말'] == 'SAT'].copy()
df_weekend_sorted = df_weekend.sort_values('복합지표', ascending=False)

print("🚨 복합 지표 Top 50 (주말)")
print("=" * 120)

top50_weekend = df_weekend_sorted.head(50)[[
    '구간', '방향', '평균소요시간_분', '평균배차간격_분', '복합지표', '측정횟수'
]].reset_index(drop=True)

display(top50_weekend)

# CSV 저장
top50_weekend.to_csv('..\data\output\복합지표_Top50_주말.csv', index=False, encoding='utf-8-sig')
print("\n✅ '복합지표_Top50_주말.csv' 파일로 저장되었습니다.")

🚨 복합 지표 Top 50 (주말)


,구간,방향,평균소요시간_분,평균배차간격_분,복합지표,측정횟수
0,전곡 → 연천,UP,7.50,55.900000,419.250000,19
1,청산 → 소요산,DOWN,5.50,55.714286,306.428571,19
2,전곡 → 청산,DOWN,4.50,55.714286,250.714286,19
3,소요산 → 동두천,DOWN,3.50,55.714286,195.000000,19
4,청산 → 전곡,UP,3.00,55.900000,167.700000,19
5,온양온천 → 신창,DOWN,4.50,31.538462,141.923077,30
6,소요산 → 청산,UP,4.50,29.878378,134.452703,19
7,배방 → 온양온천,DOWN,4.00,31.538462,126.153846,30
8,온양온천 → 배방,UP,4.00,31.156250,124.625000,35
9,천안 → 두정,UP,4.00,31.156250,124.625000,34



✅ '복합지표_Top50_주말.csv' 파일로 저장되었습니다.


## 7. 주중/주말 비교 분석

In [8]:
# 주중/주말 비교를 위한 피벗
comparison = df_combined.pivot_table(
    index=['구간', '방향'],
    columns='주중주말',
    values=['평균소요시간_분', '평균배차간격_분', '복합지표'],
    aggfunc='mean'
).round(2)

# 컬럼 flatten
comparison.columns = ['_'.join(col).strip() for col in comparison.columns.values]
comparison = comparison.reset_index()

# 주중-주말 차이 계산
if '복합지표_DAY' in comparison.columns and '복합지표_SAT' in comparison.columns:
    comparison['복합지표_차이(주중-주말)'] = comparison['복합지표_DAY'] - comparison['복합지표_SAT']
    
    # 차이가 큰 순서로 정렬 (주중이 더 나쁜 구간)
    comparison_sorted = comparison.sort_values('복합지표_차이(주중-주말)', ascending=False)
    
    print("📊 주중/주말 복합 지표 차이 Top 30 (주중이 더 나쁜 구간)")
    print("=" * 120)
    print("양수 = 주중이 더 나쁨 (소요시간↑ or 배차간격↑)")
    print("음수 = 주말이 더 나쁨")
    print("=" * 120)
    
    display(comparison_sorted.head(30))
    
    # CSV 저장
    comparison_sorted.to_csv('..\data\output\복합지표_주중주말비교.csv', index=False, encoding='utf-8-sig')
    print("\n✅ '복합지표_주중주말비교.csv' 파일로 저장되었습니다.")

📊 주중/주말 복합 지표 차이 Top 30 (주중이 더 나쁜 구간)
양수 = 주중이 더 나쁨 (소요시간↑ or 배차간격↑)
음수 = 주말이 더 나쁨


,구간,방향,복합지표_DAY,복합지표_END,복합지표_SAT,평균배차간격_분_DAY,평균배차간격_분_END,평균배차간격_분_SAT,평균소요시간_분_DAY,평균소요시간_분_END,평균소요시간_분_SAT,복합지표_차이(주중-주말)
180,천안 → 두정,UP,127.29,124.62,124.62,31.59,31.16,31.16,4.03,4.0,4.0,2.67
144,온양온천 → 배방,UP,126.34,124.62,124.62,31.59,31.16,31.16,4.00,4.0,4.0,1.72
107,소요산 → 청산,UP,136.16,134.45,134.45,30.26,29.88,29.88,4.50,4.5,4.5,1.71
72,배방 → 탕정,UP,94.76,93.47,93.47,31.59,31.16,31.16,3.00,3.0,3.0,1.29
187,탕정 → 아산,UP,78.97,77.89,77.89,31.59,31.16,31.16,2.50,2.5,2.5,1.08
62,동인천 → 인천,DOWN,37.43,36.52,36.52,10.69,10.44,10.44,3.50,3.5,3.5,0.91
81,봉명 → 천안,UP,63.17,62.31,62.31,31.59,31.16,31.16,2.00,2.0,2.0,0.86
126,아산 → 쌍용,UP,63.17,62.31,62.31,31.59,31.16,31.16,2.00,2.0,2.0,0.86
124,쌍용 → 봉명,UP,63.17,62.31,62.31,31.59,31.16,31.16,2.00,2.0,2.0,0.86
88,서울역 → 남영,DOWN,14.14,13.43,13.43,5.65,5.37,5.37,2.50,2.5,2.5,0.71



✅ '복합지표_주중주말비교.csv' 파일로 저장되었습니다.


## 8. 시각화: 복합 지표 Top 30 (주중)

In [9]:
# Top 30 구간
top30_viz = df_weekday_sorted.head(30).copy()

fig = px.bar(
    top30_viz,
    x='복합지표',
    y='구간',
    orientation='h',
    title='복합 지표 Top 30 (주중)<br><sub>소요시간 × 배차간격 = 승객 대기 시간</sub>',
    labels={'복합지표': '복합 지표 (분²)', '구간': '역 구간'},
    color='복합지표',
    color_continuous_scale='Reds',
    text='복합지표',
    hover_data={'평균소요시간_분': ':.2f', '평균배차간격_분': ':.2f'}
)

fig.update_traces(
    texttemplate='%{text:.1f}',
    textposition='outside'
)

fig.update_layout(
    height=900,
    yaxis={'categoryorder': 'total ascending'},
    showlegend=False
)

fig.show()

## 9. 시각화: 소요시간 vs 배차간격 산점도

In [10]:
# 주중 데이터로 산점도
fig = px.scatter(
    df_weekday,
    x='평균소요시간_분',
    y='평균배차간격_분',
    size='복합지표',
    color='복합지표',
    hover_data=['구간', '방향'],
    title='소요시간 vs 배차간격 (주중)<br><sub>크기와 색 = 복합지표</sub>',
    labels={
        '평균소요시간_분': '평균 소요시간 (분)',
        '평균배차간격_분': '평균 배차간격 (분)'
    },
    color_continuous_scale='Reds'
)

# 사분면 표시
median_travel = df_weekday['평균소요시간_분'].median()
median_headway = df_weekday['평균배차간격_분'].median()

fig.add_hline(y=median_headway, line_dash="dash", line_color="gray", opacity=0.5)
fig.add_vline(x=median_travel, line_dash="dash", line_color="gray", opacity=0.5)

# 사분면 주석
fig.add_annotation(
    x=df_weekday['평균소요시간_분'].max() * 0.9,
    y=df_weekday['평균배차간격_분'].max() * 0.9,
    text="⚠️ 최악<br>(길다+드물다)",
    showarrow=False,
    bgcolor="rgba(255,0,0,0.1)"
)

fig.add_annotation(
    x=df_weekday['평균소요시간_분'].min() * 1.5,
    y=df_weekday['평균배차간격_분'].min() * 1.5,
    text="✅ 최적<br>(짧다+자주)",
    showarrow=False,
    bgcolor="rgba(0,255,0,0.1)"
)

fig.show()

print(f"\n📊 사분면 분석:")
print(f"  중앙값 - 소요시간: {median_travel:.2f}분")
print(f"  중앙값 - 배차간격: {median_headway:.2f}분")
print(f"\n  우상단(최악): 소요시간↑ + 배차간격↑")
print(f"  좌하단(최적): 소요시간↓ + 배차간격↓")


📊 사분면 분석:
  중앙값 - 소요시간: 2.00분
  중앙값 - 배차간격: 10.36분

  우상단(최악): 소요시간↑ + 배차간격↑
  좌하단(최적): 소요시간↓ + 배차간격↓


## 10. 시각화: 복합 지표 분포

In [11]:
# 주중/주말 복합 지표 분포 비교
fig = px.histogram(
    df_combined,
    x='복합지표',
    color='주중주말',
    nbins=50,
    title='복합 지표 분포 (주중 vs 주말)',
    labels={'복합지표': '복합 지표 (분²)', 'count': '빈도'},
    barmode='overlay',
    opacity=0.7
)

# 평균선 추가
for day_type, color in [('DAY', 'blue'), ('SAT', 'red')]:
    if day_type in df_combined['주중주말'].values:
        mean_val = df_combined[df_combined['주중주말'] == day_type]['복합지표'].mean()
        fig.add_vline(
            x=mean_val,
            line_dash="dash",
            line_color=color,
            annotation_text=f"{day_type} 평균: {mean_val:.1f}",
            annotation_position="top"
        )

fig.show()

print("\n📊 복합 지표 통계:")
display(df_combined.groupby('주중주말')['복합지표'].describe().round(2))


📊 복합 지표 통계:


,count,mean,std,min,25%,50%,75%,max
주중주말,,,,,,,,
DAY,197.0,36.93,43.99,5.49,15.26,20.80,44.96,367.21
END,198.0,38.93,48.78,5.42,15.63,20.93,47.82,419.25
SAT,198.0,38.93,48.78,5.42,15.63,20.93,47.82,419.25


## 11. 방향별 분석 (상행 vs 하행)

In [12]:
# 방향별 복합 지표 비교 (주중)
direction_stats = df_weekday.groupby('방향').agg({
    '평균소요시간_분': 'mean',
    '평균배차간격_분': 'mean',
    '복합지표': 'mean',
    '구간': 'count'
}).round(2)
direction_stats.columns = ['평균_소요시간', '평균_배차간격', '평균_복합지표', '구간수']

print("🔀 방향별 복합 지표 비교 (주중):")
display(direction_stats)

# 방향별 Top 10
print("\n⬆️ 상행(UP) Top 10:")
up_top10 = df_weekday[df_weekday['방향'] == 'UP'].nlargest(10, '복합지표')[[
    '구간', '평균소요시간_분', '평균배차간격_분', '복합지표'
]]
display(up_top10)

print("\n⬇️ 하행(DOWN) Top 10:")
down_top10 = df_weekday[df_weekday['방향'] == 'DOWN'].nlargest(10, '복합지표')[[
    '구간', '평균소요시간_분', '평균배차간격_분', '복합지표'
]]
display(down_top10)

🔀 방향별 복합 지표 비교 (주중):


,평균_소요시간,평균_배차간격,평균_복합지표,구간수
방향,,,,
DOWN,2.46,13.24,37.31,100
UP,2.40,13.05,36.53,97



⬆️ 상행(UP) Top 10:


,구간,평균소요시간_분,평균배차간격_분,복합지표
469,전곡 → 연천,7.50,48.961538,367.211538
556,청산 → 전곡,3.00,48.923077,146.769231
320,소요산 → 청산,4.50,30.256757,136.155405
541,천안 → 두정,4.03,31.586207,127.292414
433,온양온천 → 배방,4.00,31.586207,126.344828
296,성환 → 평택,6.49,18.056452,117.186371
215,배방 → 탕정,3.00,31.586207,94.758621
119,덕정 → 지행,4.50,18.666667,84.000000
188,두정 → 직산,4.49,18.056452,81.073468
562,탕정 → 아산,2.50,31.586207,78.965517



⬇️ 하행(DOWN) Top 10:


,구간,평균소요시간_분,평균배차간격_분,복합지표
553,청산 → 소요산,5.50,49.000000,269.500000
472,전곡 → 청산,4.50,49.000000,220.500000
317,소요산 → 동두천,3.50,49.000000,171.500000
436,온양온천 → 신창,4.50,29.375000,132.187500
565,평택 → 성환,6.49,18.508333,120.119083
212,배방 → 온양온천,4.00,29.375000,117.500000
559,탕정 → 배방,3.00,29.375000,88.125000
113,덕계 → 양주,4.50,18.872881,84.927966
517,지행 → 덕정,4.49,18.872881,84.739237
523,직산 → 두정,4.50,18.508333,83.287500


## 12. 시간대별 분석 (시간대별 배차 간격 변화)

In [13]:
# 시간대별 배차 간격 계산을 위해 원본 데이터 사용
df_time = df_all[df_all['열차도착시간_dt'].notna()].copy()
df_time['시간대'] = df_time['열차도착시간_dt'].dt.hour

# 주요 역의 시간대별 배차 간격
major_stations = ['서울역', '영등포', '수원', '청량리']

for station in major_stations:
    if station in df_time['역사명'].values:
        station_data = df_time[
            (df_time['역사명'] == station) & 
            (df_time['주중주말'] == 'DAY')
        ].copy()
        
        if len(station_data) > 0:
            # 방향별로 그룹화
            hourly_headway = []
            
            for direction, group in station_data.groupby('방향'):
                group = group.sort_values('열차도착시간_dt')
                group['배차간격'] = group['열차도착시간_dt'].diff().dt.total_seconds() / 60
                
                hourly = group.groupby('시간대')['배차간격'].mean().reset_index()
                hourly['방향'] = direction
                hourly_headway.append(hourly)
            
            if hourly_headway:
                hourly_df = pd.concat(hourly_headway)
                
                fig = px.line(
                    hourly_df,
                    x='시간대',
                    y='배차간격',
                    color='방향',
                    title=f'{station} 시간대별 평균 배차간격 (주중)',
                    labels={'시간대': '시간 (시)', '배차간격': '평균 배차간격 (분)'},
                    markers=True
                )
                
                # 출퇴근 시간대 강조
                fig.add_vrect(x0=7, x1=9, fillcolor="red", opacity=0.1, 
                            annotation_text="출근", annotation_position="top left")
                fig.add_vrect(x0=18, x1=20, fillcolor="blue", opacity=0.1, 
                            annotation_text="퇴근", annotation_position="top left")
                
                fig.show()

## 13. 개선 우선순위 구간 선정

복합 지표가 높은 구간을 개선 대상으로 선정합니다.

In [14]:
# 개선 우선순위 점수 계산
df_priority = df_weekday.copy()

# 정규화 (0~100 스케일)
df_priority['소요시간_점수'] = (df_priority['평균소요시간_분'] - df_priority['평균소요시간_분'].min()) / \
                            (df_priority['평균소요시간_분'].max() - df_priority['평균소요시간_분'].min()) * 100

df_priority['배차간격_점수'] = (df_priority['평균배차간격_분'] - df_priority['평균배차간격_분'].min()) / \
                            (df_priority['평균배차간격_분'].max() - df_priority['평균배차간격_분'].min()) * 100

df_priority['복합지표_점수'] = (df_priority['복합지표'] - df_priority['복합지표'].min()) / \
                            (df_priority['복합지표'].max() - df_priority['복합지표'].min()) * 100

# 종합 점수 (가중 평균)
df_priority['종합점수'] = (
    df_priority['소요시간_점수'] * 0.3 +
    df_priority['배차간격_점수'] * 0.3 +
    df_priority['복합지표_점수'] * 0.4
).round(2)

# 개선 방안 추천
def recommend_improvement(row):
    if row['배차간격_점수'] > row['소요시간_점수']:
        return '배차 증편 우선'
    elif row['소요시간_점수'] > row['배차간격_점수']:
        return '선로/신호 개선 우선'
    else:
        return '종합 개선 필요'

df_priority['개선방안'] = df_priority.apply(recommend_improvement, axis=1)

# Top 30 개선 우선순위
priority_list = df_priority.nlargest(30, '종합점수')[[
    '구간', '방향', '평균소요시간_분', '평균배차간격_분', '복합지표',
    '소요시간_점수', '배차간격_점수', '종합점수', '개선방안'
]].reset_index(drop=True)

print("🎯 개선 우선순위 Top 30 (주중)")
print("=" * 120)
print("종합점수 = 소요시간(30%) + 배차간격(30%) + 복합지표(40%)")
print("=" * 120)
display(priority_list)

# CSV 저장
priority_list.to_csv('..\data\output\개선_우선순위_Top30.csv', index=False, encoding='utf-8-sig')
print("\n✅ '개선_우선순위_Top30.csv' 파일로 저장되었습니다.")

# 개선 방안별 통계
print("\n📊 개선 방안별 구간 수:")
display(priority_list['개선방안'].value_counts())

🎯 개선 우선순위 Top 30 (주중)
종합점수 = 소요시간(30%) + 배차간격(30%) + 복합지표(40%)


,구간,방향,평균소요시간_분,평균배차간격_분,복합지표,소요시간_점수,배차간격_점수,종합점수,개선방안
0,전곡 → 연천,UP,7.50,48.961538,367.211538,100.000000,99.912521,99.97,선로/신호 개선 우선
1,청산 → 소요산,DOWN,5.50,49.000000,269.500000,69.230769,100.000000,79.96,배차 증편 우선
2,전곡 → 청산,DOWN,4.50,49.000000,220.500000,53.846154,100.000000,69.93,배차 증편 우선
3,소요산 → 동두천,DOWN,3.50,49.000000,171.500000,38.461538,100.000000,59.90,배차 증편 우선
4,청산 → 전곡,UP,3.00,48.923077,146.769231,30.769231,99.825042,54.80,배차 증편 우선
5,소요산 → 청산,UP,4.50,30.256757,136.155405,53.846154,57.369424,47.81,배차 증편 우선
6,평택 → 성환,DOWN,6.49,18.508333,120.119083,84.461538,30.648218,47.21,선로/신호 개선 우선
7,온양온천 → 신창,DOWN,4.50,29.375000,132.187500,53.846154,55.363912,46.77,배차 증편 우선
8,성환 → 평택,UP,6.49,18.056452,117.186371,84.461538,29.620436,46.58,선로/신호 개선 우선
9,천안 → 두정,UP,4.03,31.586207,127.292414,46.615385,60.393192,45.57,배차 증편 우선



✅ '개선_우선순위_Top30.csv' 파일로 저장되었습니다.

📊 개선 방안별 구간 수:


개선방안
선로/신호 개선 우선    15
배차 증편 우선       15
Name: count, dtype: int64